In [1]:
from xml.dom import minidom
k = 0
p = s = c = n = a = 0


with open("XML-IMO.xml", 'r', encoding='utf-8') as f:
    xml = minidom.parse(f)
    Listing = xml.getElementsByTagName("Listing")
    print(Listing[0])

<DOM Element: Listing at 0x1b8dd9e9fc0>


In [ ]:
    def ignore_NoneType(argument):
        try:
            argument
        except AttributeError:
            raise Exception('ERROOOOO')
        else:
            return argument


    for count, tag in enumerate(Listing):
            for count_inside, local in enumerate(tag.getElementsByTagName("Location")):
                if not count_inside:
                    # print(local.getElementsByTagName('Country')[0].firstChild.data, k)
                    # print(local.getElementsByTagName('State')[0].firstChild.data, k)
                    # print(local.getElementsByTagName('City')[0].firstChild.data, k)
                    # print(local.getElementsByTagName('Neighborhood')[0].firstChild.data, k)
                    print(ignore_NoneType(local.getElementsByTagName('PostalCode')[0].firstChild.data), k)
                    # print(local.getElementsByTagName('Address')[0].firstChild.data, k)
                k += 1
